In [ ]:
import uuid
from IPython.core.display import display, HTML

import json

class RenderJSON:
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from renderer import render, SubfigureRenderer
import os
from benchart import BenchArt, Run
from metadata import RunMetadata
from loader import Loader

def discard_large_read(all_data):
    large_read = all_data['metadata']['benchmark'].get('large_read', None)
    if large_read is None or large_read == '' or large_read == 'none':
        return False
    return True

def discard_not_large_read(all_data):
    large_read = all_data['metadata']['benchmark'].get('large_read', None)
    if large_read is None or large_read == '' or large_read == 'none':
        return True
    return False

def discard_most(all_data):
    hostname = all_data['metadata']['machine']['instance']['hostinfo']['Hostname']
    if hostname != 'vm-6':
        return True
    return False

def discard_ultra(all_data):
    disk_kind = all_data['metadata']['machine']['disk']['limits']['size']
    return disk_kind == 'ultra1024'

def discard_not_ultra(all_data):
    disk_kind = all_data['metadata']['machine']['disk']['limits']['size']
    return disk_kind != 'ultra1024'

def data_def(all_data):
    return all_data['data']['pgbench']['progress']

def metadata_def(all_data):
    return all_data['metadata']


loader = Loader('run_data2')
loader.discard(discard_large_read)
# loader.discard(discard_not_large_read)

# loader.discard(discard_not_ultra)
# loader.discard(discard_most)
loader.discard(discard_ultra)
runs = loader.run(data_def, metadata_def)

benchart = BenchArt(runs)

relabels = {
    'machine_instance_hostinfo_Hostname': 'host',
    'benchmark_config_scale': 'scale',
    'machine_instance_hostinfo_KernelRelease': 'kernel',
    'machine_disk_block_device_settings_nr_hw_queues': '#hwq',
    'machine_disk_block_device_settings_nr_requests': 'nr_requests',
    'machine_disk_block_device_settings_queue_depth': 'queue_depth',
    'machine_disk_block_device_settings_scheduler': 'scheduler',
    'postgres.gucs.set_gucs.huge_pages': 'huge_pages',
    'machine_disk_limits_size': 'disk_kind',
    'machine_disk_size_gb': 'disk_size_gb',
    'machine_instance_hostinfo_Hostname': 'host',
    'machine_instance_type': 'instance_type',
    'benchmark_config_scale': 'pgbench_scale',
}

figure = plt.figure(figsize=(15,80), constrained_layout=True)
figure.suptitle('pgbench')
figure.supylabel('tps', fontsize=16)
disk_identifier = ['machine_disk_limits_burst_iops',
                   'machine_disk_limits_iops', 'machine_disk_limits_size',
                   'machine_disk_limits_burst_bw_mbps',
                   'machine_disk_limits_bw_mbps', 'machine_disk_size_gb', ]

benchart.part(SubfigureRenderer(relabels), 'machine_instance_hostinfo_Hostname',
              'machine_instance_type', *disk_identifier)


benchart.part(SubfigureRenderer(relabels), 'benchmark_config_scale')
# benchart.part(SubfigureRenderer(relabels), 'benchmark_large_read')

benchart.ignore('machine_instance_hostinfo_KernelRelease',
                'machine_disk_block_device_settings_nr_hw_queues',
                'machine_disk_block_device_settings_nr_requests',
                'machine_disk_block_device_settings_queue_depth',
                'machine_disk_block_device_settings_scheduler',
                'postgres.gucs.set_gucs.huge_pages')

result = render(benchart, figure, relabels)
RenderJSON(result)